In [1]:
import os
import gc
import cv2
import time
import tqdm
import random
import collections
import numpy as np
import pandas as pd
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt
import tqdm.notebook as tq
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

from torch.nn import ConvTranspose2d
from torch.nn import Conv2d
from torch.nn import MaxPool2d
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn import ReLU
from torchvision.transforms import CenterCrop
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam

from torchmetrics import JaccardIndex
import threading

import json
import cv2

from semseg import show_models
from semseg.models import *

from whuvid_dataset import WhuvidDataset
from pathlib import Path

import sys
sys.path.append('./sort') # scikit-image filterpy
import sort

from collections import defaultdict

# from deep_sort_realtime.deepsort_tracker import DeepSort

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [3]:
transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    # transforms.RandomRotation(10),
    # transforms.RandomResizedCrop(size=224, scale=(0.4, 1.0)
    #                             #  , ratio=(0.5, 2)
    #                              ),
    # resize to 224x224
    #transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [4]:
sequence = "31"
whuvid_base_path = "/home/thiago/Workspace/motion-segmentation/datasets/WHUVID"
whuvid_dataset = WhuvidDataset(whuvid_base_path, [sequence], transform, segmentation=False, flow=False, use_gdino=False)

# random_index = random.randint(0, len(whuvid_dataset))

# img, flow, imu, mask = whuvid_dataset[random_index]
# height, width = img.shape[1:]
# # plot
# fig, axes = plt.subplots(1, 3, figsize=(15, 5))
# axes[0].imshow(np.transpose(img, (1, 2, 0)))
# axes[1].imshow(mask[0], cmap='gray') 
# axes[2].imshow(np.transpose(flow, (1, 2, 0)))
# plt.show()

Sequence has 6470 images


In [5]:
len(whuvid_dataset)

6470

In [6]:
from boxmot import DeepOCSORT, StrongSORT, BoTSORT

In [7]:
tracker = BoTSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'), # which ReID model to use
    device='cuda:0',
    fp16=False,
    new_track_thresh=0.3,
    track_high_thresh=0.4,
    track_low_thresh=0.1,
    match_thresh=0.5
)

2024-11-01 19:22:58.449 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v10.0.83 🚀 Python-3.12.7 torch-2.2.2+cu121
CUDA:0 (NVIDIA GeForce RTX 4080, 15952MiB)
2024-11-01 19:22:58.490 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:183 - Loaded pretrained weights from osnet_x0_25_msmt17.pt


In [8]:
vit_path = "/home/thiago/Workspace/motion-segmentation/src/ViT-B-32.pt"
# tracker = DeepSort(max_age=5, embedder_gpu=True, embedder='clip_ViT-B/32')
# tracker = DeepSort(max_age=100, embedder='clip_ViT-B/32', embedder_wts=vit_path, embedder_gpu=True)

obj_path = os.path.join(whuvid_base_path, sequence, "other_files", "objects.json")
if os.path.exists(obj_path):
    with open(obj_path, 'r') as f:
        objects = json.load(f)
    # convert keys to int
    objects = {int(k): 
                {int(k2): v2 for k2, v2 in v.items() if v2 is not None}
                for k, v in objects.items()}
    print(f"Loaded {len(objects)} objects")
else:
    # objects = defaultdict(lambda: defaultdict(lambda: [None, None]))
    objects = defaultdict(lambda: {})

    # run full sort inference
    for i in tq.tqdm(range(len(whuvid_dataset))):
    # for i in tq.tqdm(range(500)):
        img_path = whuvid_dataset.images[i]
        img = cv2.imread(img_path)
        bbs = whuvid_dataset.bounding_boxes[i]

        # print(f"bbs: {bbs}")

        bbs_deepsort = []
        for bb in bbs:
            x1, y1, x2, y2, confidence, name = bb
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # to ((left,top,w,h), confidence, name)
            # bbs_deepsort.append([(x1, y1, x2-x1, y2-y1), confidence, name]) 
            bbs_deepsort.append([x1, y1, x2, y2, confidence, 0]) 

        if len(bbs) == 0:
            # tracked = tracker.update_tracks(bbs_deepsort, frame=img)
            # tracked = tracker.update(np.array(bbs_deepsort),img)
            tracked = tracker.update(np.empty((0, 6)),img)
        else:
            # tracked = tracker.update_tracks(bbs_deepsort, frame=img)
            tracked = tracker.update(np.array(bbs_deepsort),img)

        for t in tracked:
            # if not t.is_confirmed():
            #     continue
            # print(t)
            # id = t.track_id
            # # ltrb = t.to_ltrb(orig=True, orig_strict=True)
            # ltrb = t.to_ltrb(orig=False)
            # if ltrb is not None:
            #     bb = ltrb.astype(int).tolist()
            #     objects[id][i] = [bb, None]
            x1, y1, x2, y2, id, _, _, _ = t
            bb = [x1, y1, x2, y2]
            objects[int(id)][i] = [bb, None]
    print(f"Tracked {len(objects)} objects")
    with open(obj_path, 'w') as f:
        json.dump(objects, f)

Loaded 223 objects


In [9]:
# start at false for every object
delete_object = {k: False for k in objects.keys()}

In [10]:
new_click = False
# click_is_left = False
click_button = "left"
def get_mouse_clicks(event, x, y, flags, param):
    global mouseX, mouseY, new_click, click_button
    # if left or middle
    if event == cv2.EVENT_LBUTTONDOWN or event == cv2.EVENT_RBUTTONDOWN or event == cv2.EVENT_MBUTTONDOWN:
        mouseX, mouseY = x, y
        new_click = True
    # click_is_left = event == cv2.EVENT_LBUTTONDOWN
    if event == cv2.EVENT_LBUTTONDOWN:
        click_button = "left"
    elif event == cv2.EVENT_RBUTTONDOWN:
        click_button = "right"
    elif event == cv2.EVENT_MBUTTONDOWN:
        click_button = "middle"

def clicks_inside_bb(x, y, tracked_bbs):
    bb_clicks = []
    for id, bb, state in tracked_bbs:
        if bb is None:
            continue
        x1, y1, x2, y2 = [int(v) for v in bb]
        if x1 <= x <= x2 and y1 <= y <= y2:
            bb_clicks.append(id)
    return bb_clicks

# find most recent frame that has a state
# same state as that. If never has state, return False
def is_in_motion(id, i, objects):
    for j in range(i, -1, -1):
        if j in objects[id]:
            state = objects[id][j][1]
            if state is not None:
                return state
    return False

# ex: 0-True, 1-True, 2-None, 3-None, return 2 for i == 4 or 3 or 2
def find_none_after_state(id, i, objects):
    first_frame = min(objects[id].keys())
    for j in range(i, first_frame - 1, -1):
        if j in objects[id]:
            state = objects[id][j][1] if j in objects[id] else None
            previous_state = objects[id][j-1][1] if j-1 in objects[id] else None
            if state is None and (j == first_frame or previous_state is not None):
                return j
    return first_frame

cv2.namedWindow('image')
cv2.setMouseCallback('image', get_mouse_clicks)

# traked_objects = defaultdict(defaultdict(bool))
# objects_in_motion = set()

play_video = True
i = 0
# show dataset as opencv video
bar = tq.tqdm(total=len(whuvid_dataset))
while True:
    # set bar to i
    bar.n = i
    bar.last_print_n = i
    bar.update() 
    img_path = whuvid_dataset.images[i]
    img = cv2.imread(img_path)

    if img is None:
        print(f"None image for frame {i}")
        i = min(len(whuvid_dataset), i + 1)
        continue

    tracked_bbs = [[id, data[i][0], data[i][1]] for id, data in objects.items() if i in data]

    if new_click:
        bb_selected = clicks_inside_bb(mouseX, mouseY, tracked_bbs)
        for id in bb_selected:
            if click_button == "middle":
                delete_object[id] = not delete_object[id]
            else:
                frame_to_set = find_none_after_state(id, i, objects) if (click_button == "left") else i
                # state_of_frame = objects[id][frame_to_set][1]
                # if state_of_frame is None or state is False:
                #     objects[id][frame_to_set][1] = True
                # else:
                #     objects[id][frame_to_set][1] = False
                state = is_in_motion(id, i, objects)
                # print(f"Setting id {id} to not {state} in frame {frame_to_set}")
                objects[id][frame_to_set][1] = not state
        new_click = False

    # draw
    for id, bb, state in tracked_bbs:
        if bb is None:
            print(f"None bb for id {id} in frame {i}")
        x1, y1, x2, y2 = [int(v) for v in bb]
        if delete_object[id]:
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        else:
            annotated_state = objects[id][i][1]
            if annotated_state is True:
                # write an asterisk
                cv2.putText(img, "*", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            infered_state = is_in_motion(id, i, objects)
            center_point = (int((x1 + x2) / 2), int((y1 + y2) / 2))
            if infered_state is True:
                cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
                # write object id in yellow
                # cv2.putText(img, str(id), center_point, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
            else:
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                # write object id in yellow
                # cv2.putText(img, str(id), center_point, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
    # show
    cv2.imshow('image', img)

    key = cv2.waitKey(1) & 0xFF
    # if key != 255:
    #     print(f"key: {key}")
    if key == ord('q'):
        break
    elif key == ord(' '):
        play_video = not play_video
    # elif key == 81:
    elif key == ord('a'):
        i = max(0, i - 1)
        img = cv2.imread(whuvid_dataset.images[i])
        if img is None:
            i = max(0, i - 1)
    # elif key == 83:
    elif key == ord('d'):
        i = min(len(whuvid_dataset), i + 1)
        img = cv2.imread(whuvid_dataset.images[i])
        if img is None:
            i = min(len(whuvid_dataset), i + 1)

    if play_video:
        i = min(len(whuvid_dataset), i + 1)
    else:
        pass
# close
cv2.destroyAllWindows()

  0%|          | 0/6470 [00:00<?, ?it/s]

In [ ]:
# use is_in_motion to infer state of all frames
obj_inferred = objects.copy()
for id, data in objects.items():
    if delete_object[id]:
        continue
    for i in range(len(whuvid_dataset)):
        if i not in data:
            continue
        state = is_in_motion(id, i, objects)
        obj_inferred[id][i][1] = state
# invert everything
# obj_inferred = {id: {i: [bb, not state] for i, (bb, state) in data.items()} for id, data in obj_inferred.items()}

In [ ]:
obj_path_inf = os.path.join(whuvid_base_path, sequence, "other_files", "objects_inferred.json")
with open(obj_path_inf, 'w') as f:
    json.dump(obj_inferred, f)